In [1]:
! pip install git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-snh787ni
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-snh787ni
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=8932ca65a99430c238d8123e27d21a4e7b86072f32febe0672b1eb7e41cae2f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-qhyd4bxv/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

## Divergent reduction

This first reduction implement a vector reduction where only the pair threads are working together.

In [6]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define THREAD_NB 1024
#define SIZE 20

#define CUDA_ERROR_CHECK

#define CudaSafeCall( err ) __cudaSafeCall( err, __FILE__, __LINE__ )
#define CudaCheckError()    __cudaCheckError( __FILE__, __LINE__ )

/********************** kernel **************************/

__global__ void reduction(int * tab_in, int * tab_out){
  __shared__ int shared_mem[THREAD_NB];
  
  /*Each thread loads one element from tab_in memory to shared_memory*/
  int thread_id = threadIdx.x;
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  shared_mem[thread_id] = tab_in[index];
  __syncthreads();

  for(int i=1; i<blockDim.x; i*=2){
      if(thread_id % (2*i) == 0){
        shared_mem[thread_id] += shared_mem[thread_id+i];
      } 
      __syncthreads(); 
  }

  if (thread_id == 0){
    tab_out[blockIdx.x] = shared_mem[0];
  }
}

/********************** Host functions **************************/

/*Initialization of array */
void init_tab(int *tab, int value, int len){
    for(int i=0; i<len; i++){
        tab[i] = value;
    }
}

/*For debugging purpose*/
void affiche_tab(char *chaine, int *tab, int len){
  int k;
  int affiche = 10;
  if(len<20){
    affiche = len;
  }
  printf("\nLes %i premiers de %s: \n", affiche, chaine);
  for (k=0; k<affiche; k++) 
    printf("%i ",tab[k]);
  printf("\nLes %i derniers: \n", affiche);
  for (k=len-affiche; k<len; k++) 
    printf("%i ",tab[k]);
  printf("\n");
}

/*CUDA error checking */
inline void __cudaSafeCall( cudaError err, const char *file, const int line )
{
#ifdef CUDA_ERROR_CHECK
    if ( cudaSuccess != err )
    {
        fprintf( stderr, "cudaSafeCall() failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }
#endif

    return;
}

inline void __cudaCheckError( const char *file, const int line )
{
#ifdef CUDA_ERROR_CHECK
    cudaError err = cudaGetLastError();
    if ( cudaSuccess != err )
    {
        fprintf( stderr, "cudaCheckError() failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }

    // More careful checking. However, this will affect performance.
    // Comment away if needed.
    err = cudaDeviceSynchronize();
    if( cudaSuccess != err )
    {
        fprintf( stderr, "cudaCheckError() with sync failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }
#endif

    return;
}

/********************** main **************************/
int main(void)
{
  int *tab_in, *tab_out, *gpu_tab_in, *gpu_tab_out;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100; 
  int dim_tab = THREAD_NB * SIZE;
  int result = 0;
 
  /*Memory allocation for CPU */
  tab_in = (int*) malloc(dim_tab * sizeof(int));
  tab_out = (int*) malloc(SIZE * sizeof(int));

  /*Memory allocation for GPU */
  cudaMalloc((void**) &gpu_tab_in, dim_tab* sizeof(int));
  cudaMalloc((void**) &gpu_tab_out, SIZE * sizeof(int));  
 
  /* Initialisation of tab_in and tab_out*/
  init_tab(tab_in, 1, dim_tab);
  init_tab(tab_out, 0, SIZE);
  
  /*Visualize the input arrays */ 
  affiche_tab("Tab_in", tab_in, dim_tab);
  affiche_tab("Tab_out", tab_out, SIZE);
 
 /* Copy of tab_in and tab_out on GPU */
  cudaMemcpy(gpu_tab_in, tab_in, dim_tab * sizeof(int), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_tab_out, tab_out, SIZE * sizeof(int), cudaMemcpyHostToDevice) ;
  
  /* Let's compute the average computation time for the kernel to run */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Start timer */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ; cudaEventRecord(start) ;
  
    reduction<<<SIZE, THREAD_NB>>>(gpu_tab_in, gpu_tab_out);

    /*Checking CUDA errors */
    CudaCheckError();

    cudaEventRecord(stop) ; cudaEventSynchronize(stop) ; //Guarantees that the event is finished
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("\nAverage time over %i iterations for the basic reduction to run: %f ms\n", nIter, milliseconds/nIter);
  
  /* Copie of tab_out on CPU */
  cudaMemcpy(tab_out, gpu_tab_out, SIZE * sizeof(int), cudaMemcpyDeviceToHost) ;
  
  /*Visualize the output array */
  affiche_tab("Reduction out", tab_out, SIZE); 

  /*Do the actual reduction and check result*/
  for(int j=0; j<SIZE; j++){
      result += tab_out[j];
  }
 
  if(result != dim_tab)
    printf("\nThe reduction is wrong. Reduction value : %i", result);
  else
    printf("\nThe reduction is right. Reduction value : %i", result);
 

  /* Free GPU memory*/ 
  cudaFree(gpu_tab_in);
  cudaFree(gpu_tab_out);

  /* Free CPU memory*/
  free(tab_in);
  free(tab_out);
}


Les 10 premiers de Tab_in: 
1 1 1 1 1 1 1 1 1 1 
Les 10 derniers: 
1 1 1 1 1 1 1 1 1 1 

Les 10 premiers de Tab_out: 
0 0 0 0 0 0 0 0 0 0 
Les 10 derniers: 
0 0 0 0 0 0 0 0 0 0 

Average time over 100 iterations for the basic reduction to run: 0.022144 ms

Les 10 premiers de Reduction out: 
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 
Les 10 derniers: 
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 

The reduction is right. Reduction value : 20480


## Parallel reduction with bank conflicts

This second version implements a reduction where every threads are working together.

In [10]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define THREAD_NB 1024
#define SIZE 20

#define CUDA_ERROR_CHECK

#define CudaSafeCall( err ) __cudaSafeCall( err, __FILE__, __LINE__ )
#define CudaCheckError()    __cudaCheckError( __FILE__, __LINE__ )

/********************** kernel **************************/

__global__ void reduction(int * tab_in, int * tab_out){
  __shared__ int shared_mem[THREAD_NB];
  
  /*Each thread loads one element from tab_in memory to shared_memory*/
  int thread_id = threadIdx.x;
  int index_th = blockIdx.x * blockDim.x + threadIdx.x;
  shared_mem[thread_id] = tab_in[index_th];
  __syncthreads();

  for (int i=1; i < blockDim.x; i*= 2) {
    int index = 2 * i * thread_id;
    if (index < blockDim.x) {
      shared_mem[index] += shared_mem[index + i];
    }
    __syncthreads();
  }

  if (thread_id == 0){
    tab_out[blockIdx.x] = shared_mem[0];
  }
}

/********************** Host functions **************************/

/*Initialization of array */
void init_tab(int *tab, int value, int len){
    for(int i=0; i<len; i++){
        tab[i] = value;
    }
}

/*For debugging purpose*/
void affiche_tab(char *chaine, int *tab, int len){
  int k;
  int affiche = 10;
  if(len<20){
    affiche = len;
  }
  printf("\nLes %i premiers de %s: \n", affiche, chaine);
  for (k=0; k<affiche; k++) 
    printf("%i ",tab[k]);
  printf("\nLes %i derniers: \n", affiche);
  for (k=len-affiche; k<len; k++) 
    printf("%i ",tab[k]);
  printf("\n");
}

/*CUDA error checking */
inline void __cudaSafeCall( cudaError err, const char *file, const int line )
{
#ifdef CUDA_ERROR_CHECK
    if ( cudaSuccess != err )
    {
        fprintf( stderr, "cudaSafeCall() failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }
#endif

    return;
}

inline void __cudaCheckError( const char *file, const int line )
{
#ifdef CUDA_ERROR_CHECK
    cudaError err = cudaGetLastError();
    if ( cudaSuccess != err )
    {
        fprintf( stderr, "cudaCheckError() failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }

    // More careful checking. However, this will affect performance.
    // Comment away if needed.
    err = cudaDeviceSynchronize();
    if( cudaSuccess != err )
    {
        fprintf( stderr, "cudaCheckError() with sync failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }
#endif

    return;
}

/********************** main **************************/
int main(void)
{
  int *tab_in, *tab_out, *gpu_tab_in, *gpu_tab_out;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100; 
  int dim_tab = THREAD_NB * SIZE;
  int result = 0;
 
  /*Memory allocation for CPU */
  tab_in = (int*) malloc(dim_tab * sizeof(int));
  tab_out = (int*) malloc(SIZE * sizeof(int));

  /*Memory allocation for GPU */
  cudaMalloc((void**) &gpu_tab_in, dim_tab* sizeof(int));
  cudaMalloc((void**) &gpu_tab_out, SIZE * sizeof(int));  
 
  /* Initialisation of tab_in and tab_out*/
  init_tab(tab_in, 1, dim_tab);
  init_tab(tab_out, 0, SIZE);
  
  /*Visualize the input arrays */ 
  affiche_tab("Tab_in", tab_in, dim_tab);
  affiche_tab("Tab_out", tab_out, SIZE);
 
 /* Copy of tab_in and tab_out on GPU */
  cudaMemcpy(gpu_tab_in, tab_in, dim_tab * sizeof(int), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_tab_out, tab_out, SIZE * sizeof(int), cudaMemcpyHostToDevice) ;
  
  /* Let's compute the average computation time for the kernel to run */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Start timer */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ; cudaEventRecord(start) ;
  
    reduction<<<SIZE, THREAD_NB>>>(gpu_tab_in, gpu_tab_out);

    /*Checking CUDA errors */
    CudaCheckError();

    cudaEventRecord(stop) ; cudaEventSynchronize(stop) ; //Guarantees that the event is finished
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("\nAverage time over %i iterations for the parallel reduction (with Bank conflicts) to run: %f ms\n", nIter, milliseconds/nIter);
  
  /* Copie of tab_out on CPU */
  cudaMemcpy(tab_out, gpu_tab_out, SIZE * sizeof(int), cudaMemcpyDeviceToHost) ;
  
  /*Visualize the output array */
  affiche_tab("Reduction out", tab_out, SIZE); 

  /*Do the actual reduction and check result*/
  for(int j=0; j<SIZE; j++){
      result += tab_out[j];
  }
 
  if(result != dim_tab)
    printf("\nThe reduction is wrong. Reduction value : %i", result);
  else
    printf("\nThe reduction is right. Reduction value : %i", result);
 

  /* Free GPU memory*/ 
  cudaFree(gpu_tab_in);
  cudaFree(gpu_tab_out);

  /* Free CPU memory*/
  free(tab_in);
  free(tab_out);
}


Les 10 premiers de Tab_in: 
1 1 1 1 1 1 1 1 1 1 
Les 10 derniers: 
1 1 1 1 1 1 1 1 1 1 

Les 10 premiers de Tab_out: 
0 0 0 0 0 0 0 0 0 0 
Les 10 derniers: 
0 0 0 0 0 0 0 0 0 0 

Average time over 100 iterations for the parallel reduction (with Bank conflicts) to run: 0.016999 ms

Les 10 premiers de Reduction out: 
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 
Les 10 derniers: 
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 

The reduction is right. Reduction value : 20480


## Parallel reduction without without bank conflicts

In [20]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define THREAD_NB 1024
#define SIZE 20

#define CUDA_ERROR_CHECK

#define CudaSafeCall( err ) __cudaSafeCall( err, __FILE__, __LINE__ )
#define CudaCheckError()    __cudaCheckError( __FILE__, __LINE__ )

/********************** kernel **************************/

__global__ void reduction(int * tab_in, int * tab_out){
  __shared__ int shared_mem[THREAD_NB];
  
  /*Each thread loads one element from tab_in memory to shared_memory*/
  int thread_id = threadIdx.x;
  int index_th = blockIdx.x * blockDim.x + threadIdx.x;
  shared_mem[thread_id] = tab_in[index_th];
  __syncthreads();

  for (int i=blockDim.x/2; i>0; i>>=1) {
    if (thread_id < i) {
      shared_mem[thread_id] += shared_mem[thread_id + i];
    }
    __syncthreads();
  }

  if (thread_id == 0){
    tab_out[blockIdx.x] = shared_mem[0];
  }
}

/********************** Host functions **************************/

/*Initialization of array */
void init_tab(int *tab, int value, int len){
    for(int i=0; i<len; i++){
        tab[i] = value;
    }
}

/*For debugging purpose*/
void affiche_tab(char *chaine, int *tab, int len){
  int k;
  int affiche = 10;
  if(len<20){
    affiche = len;
  }
  printf("\nLes %i premiers de %s: \n", affiche, chaine);
  for (k=0; k<affiche; k++) 
    printf("%i ",tab[k]);
  printf("\nLes %i derniers: \n", affiche);
  for (k=len-affiche; k<len; k++) 
    printf("%i ",tab[k]);
  printf("\n");
}

/*CUDA error checking */
inline void __cudaSafeCall( cudaError err, const char *file, const int line )
{
#ifdef CUDA_ERROR_CHECK
    if ( cudaSuccess != err )
    {
        fprintf( stderr, "cudaSafeCall() failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }
#endif

    return;
}

inline void __cudaCheckError( const char *file, const int line )
{
#ifdef CUDA_ERROR_CHECK
    cudaError err = cudaGetLastError();
    if ( cudaSuccess != err )
    {
        fprintf( stderr, "cudaCheckError() failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }

    // More careful checking. However, this will affect performance.
    // Comment away if needed.
    err = cudaDeviceSynchronize();
    if( cudaSuccess != err )
    {
        fprintf( stderr, "cudaCheckError() with sync failed at %s:%i : %s\n",
                 file, line, cudaGetErrorString( err ) );
        exit( -1 );
    }
#endif

    return;
}

/********************** main **************************/
int main(void)
{
  int *tab_in, *tab_out, *gpu_tab_in, *gpu_tab_out;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100; 
  int dim_tab = THREAD_NB * SIZE;
  int result = 0;
 
  /*Memory allocation for CPU */
  tab_in = (int*) malloc(dim_tab * sizeof(int));
  tab_out = (int*) malloc(SIZE * sizeof(int));

  /*Memory allocation for GPU */
  cudaMalloc((void**) &gpu_tab_in, dim_tab* sizeof(int));
  cudaMalloc((void**) &gpu_tab_out, SIZE * sizeof(int));  
 
  /* Initialisation of tab_in and tab_out*/
  init_tab(tab_in, 1, dim_tab);
  init_tab(tab_out, 0, SIZE);
  
  /*Visualize the input arrays */ 
  affiche_tab("Tab_in", tab_in, dim_tab);
  affiche_tab("Tab_out", tab_out, SIZE);
 
 /* Copy of tab_in and tab_out on GPU */
  cudaMemcpy(gpu_tab_in, tab_in, dim_tab * sizeof(int), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_tab_out, tab_out, SIZE * sizeof(int), cudaMemcpyHostToDevice) ;
  
  /* Let's compute the average computation time for the kernel to run */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Start timer */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ; cudaEventRecord(start) ;
  
    reduction<<<SIZE, THREAD_NB>>>(gpu_tab_in, gpu_tab_out);

    /*Checking CUDA errors */
    CudaCheckError();

    cudaEventRecord(stop) ; cudaEventSynchronize(stop) ; //Guarantees that the event is finished
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("\nAverage time over %i iterations for the parallel reduction (without Bank conflicts) to run: %f ms\n", nIter, milliseconds/nIter);
  
  /* Copie of tab_out on CPU */
  cudaMemcpy(tab_out, gpu_tab_out, SIZE * sizeof(int), cudaMemcpyDeviceToHost) ;
  
  /*Visualize the output array */
  affiche_tab("Reduction out", tab_out, SIZE); 

  /*Do the actual reduction and check result*/
  for(int j=0; j<SIZE; j++){
      result += tab_out[j];
  }
 
  if(result != dim_tab)
    printf("\nThe reduction is wrong. Reduction value : %i", result);
  else
    printf("\nThe reduction is right. Reduction value : %i", result);
 

  /* Free GPU memory*/ 
  cudaFree(gpu_tab_in);
  cudaFree(gpu_tab_out);

  /* Free CPU memory*/
  free(tab_in);
  free(tab_out);
}


Les 10 premiers de Tab_in: 
1 1 1 1 1 1 1 1 1 1 
Les 10 derniers: 
1 1 1 1 1 1 1 1 1 1 

Les 10 premiers de Tab_out: 
0 0 0 0 0 0 0 0 0 0 
Les 10 derniers: 
0 0 0 0 0 0 0 0 0 0 

Average time over 100 iterations for the parallel reduction (without Bank conflicts) to run: 0.016377 ms

Les 10 premiers de Reduction out: 
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 
Les 10 derniers: 
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 

The reduction is right. Reduction value : 20480


## Thrust version of reduction

In [11]:
%%cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <algorithm>
#include <cstdlib>

#define THREAD_NB 1024
#define SIZE 20

int f()
{ 
    return 1;
}
/********************** main **************************/
int main(void)
{
  double milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  int dim_tab = THREAD_NB * SIZE;
  int result = 0;
  
  // generate input data
  thrust::host_vector<int> tab_in(dim_tab);
  std::generate(tab_in.begin(), tab_in.end(), f);

  // transfer to device and compute sum
  thrust::device_vector<int> tab_out = tab_in;
  cudaDeviceSynchronize();

  /* Let's compute the average computation time for the kernel to run */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Start timer */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ; cudaEventRecord(start) ;

    result = thrust::reduce(tab_out.begin(), tab_out.end(), 0, thrust::plus<int>());
    cudaDeviceSynchronize();

    cudaEventRecord(stop) ; cudaEventSynchronize(stop) ; //Guarantees that the event is finished
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("\nAverage time over %i iterations for the Thrust version of parallel reduction to run: %f ms\n", nIter, milliseconds/nIter);

  //Check result
  if(result != dim_tab)
    printf("\nThe reduction is wrong. Reduction value : %i", result);
  else
    printf("\nThe reduction is right. Reduction value : %i", result);
 
  return 0;
}


Average time over 100 iterations for the Thrust version of parallel reduction to run: 0.043295 ms

The reduction is right. Reduction value : 20480


## Comparison of results and analysis

 
 Here is the summary of the average computation time of the different reduction method : 

Reduction method  | Divergent | Parallel/Bank conflicts | Parallel/No Bank conflicts | Thrust version
--- | --- | --- | --- | ---
Time (ms) | 0.022 | 0.017 | 0.016 | 0.043

* The **divergent** version of reduction only make the paired thread to work together, so if we want to compute the sum of a n lengthed array, only n/2 thread will work. That is why this method is the slowest one.

* The **first parallel** version of reduction is a version with bank conflicts. In order to make all threads work together, we use strided index as well as non-divergent branch. This is why the the parallel reduction is quicker than the divergent one. However, as different threads are trying to read into the same region of memory (called banks) at the same time, ang given that a bank can only retrieve a cell memory per clock cycle, we cause bank conflicts. If n threads try to access n memory cells into the same bank, with this version we achieve that in n clock cycles. This is a limitation that we are solve in the second version of the parallel reduction.

* The **second parallel** version of reduction is implemented without any bank conflict, meaning that we can access n memory celles in only one clock cycle. This is achevied by using a reversed loop and thread-id based indexing. This version is the most optimized version of reduction and so the quickest.

* We also implemented a **version with the Thrust library**, that does not need particular code wirting. We just build the arrays with the C++ built-in functions, and then we launch the reduction with the reduce function of Thrust. This library handles GPU without the need to write kernel functions. However, this function has the worst average computation time.